In [1]:
# #import commands for first time you runnging the frondend:
# !pip install requests
# !pip install time
# !pip install schedule
# !pip install IPython
# !pip install ipywidgets
# !pip install pandas
# !pip install --upgrade matplotlib
# !pip install plotly
# !pip install numpy
# !pip install termcolor

In [112]:
%matplotlib inline
import time
import threading
import matplotlib.pyplot as plt
from IPython import display as ipy
from ipywidgets import HBox, VBox, Label, Button, Text, FloatSlider, Layout, interactive
import ipywidgets as widgets
import numpy as np

# const
SENSORS = [(1, "PM10: PM Freiburg City Schlossbergring", "5b8449037c519100190fc72c"),
           (2, "PM10: Basel, Leonhardsgraben", "5d76badc953683001aa283f5")]
# private
_cancel_trafficlight = False
_sensor_id = None

# widget outputs
out = widgets.Output(layout={'border': 'none'})

# controls
lbl_space = Label(layout=Layout(width='80px'))
lbl_sensors = Label("sensors")
lbl_minlimit = Label("inoffensive")
lbl_maxlimit = Label("critical")
lbl_startstop = Label("traffic light demo")
btn_start = Button(description="start", button_style="success", layout=Layout(height='auto', width='auto'))
btn_stop = Button(description="stop", button_style="danger", layout=Layout(height='auto', width='auto'))
drp_sensors = widgets.Dropdown(
    options=[(s[1], s[0]) for s in SENSORS],
    value=1)
sld_min = FloatSlider(min=15.0, max=30.000, step=0.1, value=10.000)
sld_max = FloatSlider(min=30.0, max=100.000, step=0.1, value=50.000)
sld_value = FloatSlider(min=-1.0, max=100.000, step=0.1, value=-1.000)
sld_value.layout.visibility = 'hidden'

# methods
def start_trafficlight(button):
    global drp_sensors
    global _sensor_id
    global _cancel_trafficlight
    _cancel_trafficlight = False
    _sensor_id = [s[2] for s in SENSORS if s[0] == drp_sensors.value][0]
    thread = threading.Thread(
    target=loop_trafficlight)
    thread.start()

def stop_trafficlight(button):
    global _cancel_trafficlight
    global sld_value
    _cancel_trafficlight = True
    
def get_trafficlight_color(val):
    global sld_min
    global sld_max
    min_val = sld_min.value
    max_val = sld_max.value
    if val < 0: return [["grey", 0.3], ["grey", 0.3], ["grey", 0.3]]
    if val > max_val: return [["red", 1.0], ["grey", 0.3], ["grey", 0.3]]
    if val <= max_val and val >= min_val: return [["grey", 0.3], ["orange", 1.0], ["grey", 0.3]] 
    return [["grey", 0.3], ["grey", 0.3], ["green", 1.0]] 

def show_trafficlight(v):
    global _sensor_id
    lights = get_trafficlight_color(v)
    fig, ax = plt.subplots()
    ax.set(xlim=(0.0, 1.0), ylim=(0.0, 2.0))
    ax.set_aspect('equal')
    plt.xticks([])
    plt.yticks([])
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_facecolor("black")
    ax.add_patch(plt.Circle((0.5, 1.6), .26, color=lights[0][0], alpha=lights[0][1]))
    ax.add_patch(plt.Circle((0.5, 1.0), .26, color=lights[1][0], alpha=lights[1][1]))
    ax.add_patch(plt.Circle((0.5, 0.4), .26, color=lights[2][0], alpha=lights[2][1]))
    fnt_size = 16
    if v >= 0.0: plt.xlabel("{:.2f} μg/m³".format(v), fontsize=fnt_size)
    plt.show()     
    
def loop_trafficlight():
    global cancel_trafficlight
    global sld_value
    data = [(2. * np.sin(x) + np.sin(2. * x)) * 20. + 5. for x in np.arange(0., 2*np.pi, 0.1)];    
    for val in data:
        if _cancel_trafficlight: break
        sld_value.value = val
        time.sleep(3)
    sld_value.value = -1
    
# events
btn_start.on_click(start_trafficlight)
btn_stop.on_click(stop_trafficlight)

# show time
interactive_plot_light = interactive(show_trafficlight, v=sld_value)
output_light = interactive_plot_light.children[-1]
output_light.layout.height = '284px'
start_stop = HBox([btn_start, btn_stop])
controls = VBox([
    lbl_sensors, 
    drp_sensors, 
    lbl_minlimit, 
    sld_min, 
    lbl_maxlimit, 
    sld_max, 
    lbl_startstop, 
    start_stop])

with out: ipy.display(HBox([controls, lbl_space, interactive_plot_light]))
ipy.display(out)


Output(layout=Layout(border='none'))